# ICESat-2 ATL14 Viewer
Plots ICESat-2 Gridded Land Ice Height data (ATL14) on interactive leaflet maps

#### Load necessary packages

In [ ]:
import os
import ipywidgets
import rioxarray
import IS2view
# autoreload
%load_ext autoreload
%autoreload 2

### Interactive Mapping with Leaflet

Interactive maps within IS2view are built upon [ipyleaflet](https://ipyleaflet.readthedocs.io).

There are 2 projections available for mapping ([North](https://epsg.io/3413) and [South](https://epsg.io/3031)) depending on the ATL14 region selected.

In [ ]:
IS2widgets = IS2view.widgets()
ipywidgets.VBox([
    IS2widgets.release,
    IS2widgets.region,
])

### Create ipyleaflet map in specified projection

In [ ]:
m = IS2view.leaflet(IS2widgets.projection,
    center=IS2widgets.center,
    zoom=IS2widgets.zoom,
    attribution=True)
m.map

### Query CMR for ATL14 granule
Eventually will switch this to the s3 instance when publicly available

In [ ]:
ids,urls = IS2view.utilities.cmr(product='ATL14',
    release=IS2widgets.release.value,
    regions=IS2widgets.region.value,
    resolutions='100m',
    provider='NSIDC_ECS')
if not os.access(ids[0], os.F_OK):
    IS2view.utilities.from_nsidc(urls[0], local=ids[0])

### Read and inspect ATL14 data

In [ ]:
variables = ['h', 'h_sigma', 'misfit_rms', 'misfit_scaled_rms']
ds = rioxarray.open_rasterio(ids[0],
    variable=variables,
    masked=True)
ds

In [ ]:
IS2widgets.set_variables(ds)
IS2widgets.set_atl14_defaults()
ipywidgets.VBox([
    IS2widgets.variable,
    IS2widgets.cmap,
    IS2widgets.reverse,
])

### Add xarray dataset as an image service layer

In [ ]:
ds.leaflet.plot(m.map,
    vmin=IS2widgets.vmin, vmax=IS2widgets.vmax,
    variable=IS2widgets.variable.value,
    cmap=IS2widgets.colormap,
    opacity=0.75,
    enable_popups=True)

### Remove the image service layer from the map

In [ ]:
ds.leaflet.reset()